# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey
gmaps.configure(api_key=gkey)
from config import api_key

ImportError: cannot import name 'gkey'

In [3]:
### Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame
vacation_df = pd.read_csv("../starter_code/Weather.csv")
vacation_df.dropna()


FileNotFoundError: [Errno 2] File ../starter_code/Weather.csv does not exist: '../starter_code/Weather.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:

locations = vacation_df[["Lat", "Lng"]]
# fill nan values and convert to float
humidity = vacation_df["Humidity"].astype(float)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                             dissipating=False, max_intensity=20,
                             point_radius=1)


# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
newdf = vacation_df[(vacation_df['Max Temp'] >= 70) & (vacation_df['Max Temp'] <= 80)]
newdf

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
13,Mar del Plata,20,AR,1583345015,69,-38.00,-57.56,79.00,11.41
14,Lebu,6,CL,1583345171,40,-37.62,-73.65,72.16,12.73
18,Rikitea,93,PF,1583345173,78,-23.12,-134.97,77.32,22.41
24,Arraial do Cabo,75,BR,1583345174,74,-22.97,-42.03,78.80,13.87
25,Amuntai,100,ID,1583345174,95,-2.42,115.25,73.56,1.34
26,Cape Town,20,ZA,1583345174,88,-33.93,18.42,71.01,8.05
28,Bredasdorp,1,ZA,1583345174,60,-34.53,20.04,71.60,10.29
30,Bloemhof,0,ZA,1583345175,35,-27.65,25.61,73.04,9.57
31,Avarua,1,CK,1583345175,100,-21.21,-159.78,77.00,2.24
32,Geraldton,67,AU,1583345175,83,-28.77,114.60,71.60,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = vacation_df[['Lat', 'Lng']].apply(tuple, axis=1)


In [9]:
new_df =  vacation_df["City"]


In [10]:
newer_df = pd.DataFrame(new_df)


In [11]:
# set up additional columns to hold information
newer_df['hotel name'] = ""
newer_df['address'] = ""
newer_df['price_level'] = ""
newer_df['rating'] = ""

newer_df.head()

,City,hotel name,address,price_level,rating
0,Bluff,,,,
1,Atuona,,,,
2,Aklavik,,,,
3,Longyearbyen,,,,
4,Nokaneng,,,,


In [25]:
city = []
for x in newer_df["city"]:
    city.append(x)
city = ""    
params = {
    "Location": city,
    "key": gkey
}
for index, row in newer_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    #city = row['City']
    

    # update address key value
    params['address'] = f"{city}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    cities_lat_lng=  cities_lat_lng.json()
    newer_df.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    newer_df.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
newer_df.head()
        
    

KeyError: 'city'

In [20]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": gkey
    "location": city
}

# Use the lat/lng we recovered to identify airports
for index, row in newer_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        newer_df.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        newer_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        newer_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map